In [1]:
import pandas as pd
from datetime import datetime

session_id = 145

df = pd.read_json('./data/{}/{}.json'.format(session_id, session_id))

In [2]:
# clean data
# successData = df[df['success'] != False] # remove unsuccessful data
cleanData = df.dropna(subset=['data']).reset_index(drop=True) # remove missing data due to error 
posData = cleanData['data']
posData = pd.json_normalize(posData)

# # clean up unwanted features
cleanData = cleanData.drop(['data', 'version', 'errorCode', 'success', 'alive'], axis=1)
posData = posData.drop(['anchorData'], axis=1)

res = pd.concat([cleanData,posData], axis=1)
res.head()
res.max(axis=0)

tagId                                            27263
timestamp                2021-08-10 06:09:49.900999936
coordinates.x                                     7742
coordinates.y                                     9695
coordinates.z                                     3354
orientation.yaw                                  6.282
orientation.roll                                -0.029
orientation.pitch                                3.141
metrics.latency                                  132.6
metrics.rates.update                             1.601
metrics.rates.success                              1.6
dtype: object

In [3]:
# Re-assign colour
tagIdDict = {
    27176:'GREEN',# before 318 green id is 27160
    27226:'RED',
    27209:'BLUE',
    27263:'YELLOW',
    27152:'PINK',
    27154:'T_MED',
    27234:'T_ECG'
}

relabelCol={
    "coordinates.x":"x", 
    "coordinates.y": "y",
    "coordinates.z": "z"
}

res['tagId'] = res.apply (lambda row: tagIdDict[row['tagId']], axis=1)
res

res = res.rename(columns=relabelCol)
res


,tagId,timestamp,x,y,z,orientation.yaw,orientation.roll,orientation.pitch,metrics.latency,metrics.rates.update,metrics.rates.success
0,GREEN,2021-08-10 05:36:35.727000064,2139,-1965,2528,2.305,-0.862,-0.013,92.8,0.968,0.906
1,PINK,2021-08-10 05:36:35.824000000,5865,7337,0,3.097,-0.029,0.013,97.7,0.984,0.984
2,RED,2021-08-10 05:36:35.923000064,1613,-1747,750,4.216,-1.247,3.076,73.8,0.509,0.174
3,YELLOW,2021-08-10 05:36:35.997999872,133,-506,882,2.909,-1.404,-0.151,87.8,0.883,0.640
4,BLUE,2021-08-10 05:36:36.085999872,645,-2141,3354,5.246,-1.517,-3.112,84.8,1.018,0.788
...,...,...,...,...,...,...,...,...,...,...,...
16663,PINK,2021-08-10 06:09:49.508999936,5718,7369,0,3.107,-0.032,0.012,97.7,1.329,1.329
16664,RED,2021-08-10 06:09:49.608000000,3972,2032,750,2.594,-1.270,2.388,91.8,1.335,1.289
16665,YELLOW,2021-08-10 06:09:49.701999872,6586,838,882,0.135,-1.142,0.216,97.7,1.341,1.288
16666,BLUE,2021-08-10 06:09:49.800000000,4638,8182,3354,0.649,-1.334,2.453,98.7,1.341,1.336


In [52]:
# TODO
# Example
sample_time_raw= "2021-08-10_15-36-35-417"
start_time = datetime.strptime(sample_time_raw, '%Y-%m-%d_%H-%M-%S-%f')
st = pd.to_datetime(start_time)
print(st)

res_offset = res
# convert time
res_offset['timestamp'] = res['timestamp'] + pd.DateOffset(hours=10) # AEDT +10h
# read timestamps
res_ts = res_offset[res_offset['timestamp'] >= st]
res_ts['timestamp'] = pd.to_datetime(res_ts['timestamp']).dt.normalize()
res_ts

2021-08-10 15:36:35.417000


,tagId,timestamp,x,y,z,orientation.yaw,orientation.roll,orientation.pitch,metrics.latency,metrics.rates.update,metrics.rates.success
0,GREEN,2021-08-15,2139,-1965,2528,2.305,-0.862,-0.013,92.8,0.968,0.906
1,PINK,2021-08-15,5865,7337,0,3.097,-0.029,0.013,97.7,0.984,0.984
2,RED,2021-08-15,1613,-1747,750,4.216,-1.247,3.076,73.8,0.509,0.174
3,YELLOW,2021-08-15,133,-506,882,2.909,-1.404,-0.151,87.8,0.883,0.640
4,BLUE,2021-08-15,645,-2141,3354,5.246,-1.517,-3.112,84.8,1.018,0.788
...,...,...,...,...,...,...,...,...,...,...,...
16663,PINK,2021-08-15,5718,7369,0,3.107,-0.032,0.012,97.7,1.329,1.329
16664,RED,2021-08-15,3972,2032,750,2.594,-1.270,2.388,91.8,1.335,1.289
16665,YELLOW,2021-08-15,6586,838,882,0.135,-1.142,0.216,97.7,1.341,1.288
16666,BLUE,2021-08-15,4638,8182,3354,0.649,-1.334,2.453,98.7,1.341,1.336


In [41]:
# TODO: ATM, it randomises the audio data. Use Linxuan's code to analyse from the audio
# import random
# res_ts['audio'] = [random.choice(["0", "1" ]) for k in res_ts.index]
# res_ts
colour_tagId = "RED"
res_ts['audio'] = 0

dfa = pd.read_csv('./out/sim/{}/{}_out.csv'.format(session_id, colour_tagId))
dfa['timestamp'] = pd.to_datetime(dfa['audio time'], format='%H:%M:%S')

dfa['timestamp'] = dfa['timestamp'] + start_time
# + pd.DateOffset(year=st.year, month=st.month, day = st.day, hour=st.hour, minute=st.minute, second=st.second, microsecond=st.microsecond)
dfa
# sample = pd.merge(res_ts, dfa, on='timestamp')
# sample

TypeError: cannot add DatetimeArray and Timestamp

In [16]:
# CONVERT TO CSV and ready to use
res_ts.to_csv("./clean/{}.csv".format(session_id), sep=',', encoding='utf-8', index=False)